In [16]:
import pandas as pd
train_df = pd.read_table('1. 실습용자료.txt',sep='|',encoding="cp949")
test_df = pd.read_table('2. 모델개발용자료.txt',sep='|',encoding="cp949")

In [20]:
train_df.to_csv("1. 실습용자료.csv",encoding="utf-8-sig",index=False)
test_df.to_csv("2. 모델개발용자료.csv",encoding="utf-8-sig",index=False)

(100000, 7)

In [24]:
train_df.head(3)

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매


In [27]:
test_df.head(3)

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,NaN,NaN,NaN,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,NaN,NaN,NaN,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,NaN,NaN,NaN,절에서,신도을 대상으로,불교단체운영


- text_obj, text_mthd, text_deal 3가지 텍스트 데이터를 기반으로 산업분류코드 예측하기